# Working with sentinel-2 data

## AIM

The aim is to batch clip processed sentinel imagery with the extend of the study area

## DATA MANAGMENT

In [10]:
import os
from glob import glob

# import gui tkinter
from tkinter import Tk
import tkinter.filedialog as tkfd

### Set input and output directories

**Set input directory**, storing Sentinel data, and the **output directory**, where the processed imagery wil
l be saved.

In [6]:
root = Tk()
input_dir = tkfd.askdirectory(title = "Select input folder", initialdir = "C:/")
output_dir = tkfd.askdirectory(title = "Select output folder", initialdir = "C:/")
study_area = tkfd.askopenfilename(title = "Select extend study area", 
                                  initialdir = "C:/", 
                                  filetypes = (("GeoJSON", "*.geojson"), ("all files", "*.*")))
root.destroy()

In [7]:
input_dir

'C:/SS'

In [8]:
output_dir

'C:/SSClip'

In [9]:
study_area

'C:/WERKMAP/Anaconda/sentinelsat/geojson/Sentinelsat.geojson'

### CLIPPING with STUDY AREA

In [29]:
import osgeo
from osgeo import gdal
from rasterstats import zonal_stats

In [14]:
# Enable GDAL/OGR exceptions and set GDAL data
gdal.UseExceptions()
osgeo.gdal.SetConfigOption("GDAL_DATA", "C:/Users/jeroen.dereu@inbo.be/Software/gdal-2.1.3/data")

In [51]:
for path in glob(os.path.join(input_dir, "**", "*/merged_bands*", "*.tif"), recursive=True):
    basename = os.path.basename(path)
    filename, file_extention = os.path.splitext(basename)
    date = filename[0:7]
    mean = zonal_stats(study_area, path, stats = "mean", nodata = 0)
    if mean == [{'mean': None}]:
        pass
    else:
        if not os.path.isdir(output_dir + "/merged_bands"):
            os.mkdir(output_dir + "/merged_bands")
        if not os.path.isdir(output_dir + "/merged_bands" + "/" + date):
            os.mkdir(output_dir + "/merged_bands" + "/" + date)    
        output_folder = output_dir + "/merged_bands" + "/" + date + "/"
        output_file = output_folder + filename + ".tif"
        gdal.Warp(destNameOrDestDS = output_file, 
                  srcDSOrSrcDSTab = path, 
                  cropToCutline = True, 
                  cutlineDSName = study_area)
for path in glob(os.path.join(input_dir, "**", "*/stacked*", "*.tif"), recursive=True):
    basename = os.path.basename(path)
    filename, file_extention = os.path.splitext(basename)
    date = filename[0:7]
    mean = zonal_stats(study_area, path, stats = "mean", nodata = 0)
    if mean == [{'mean': None}]:
        pass
    else:
        if not os.path.isdir(output_dir + "/stacked"):
            os.mkdir(output_dir + "/stacked")
        if not os.path.isdir(output_dir + "/stacked" + "/" + date):
            os.mkdir(output_dir + "/stacked" + "/" + date)    
        output_folder = output_dir + "/stacked" + "/" + date + "/"
        output_file = output_folder + filename + ".tif"
        gdal.Warp(destNameOrDestDS = output_file, 
                  srcDSOrSrcDSTab = path, 
                  cropToCutline = True, 
                  cutlineDSName = study_area)